In [4]:
from audioDatastore.AudioDatastore import AudioDatastore, subset
import os.path

In [3]:

datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\speech_commands_v0.01"

ads = AudioDatastore()
ads.populate(datasetFolder,include_sub_folders=True, label_source=True)

In [5]:
ads_subset = subset(ads, label='stop')

speakers = []
for file in ads_subset.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

ads_subset.set(labels=speakers)